In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict
import pickle
from IPython import display
%matplotlib inline

In [2]:
def get_player_map(outcome):    
    # get unique players
    players = set()
    account_id_cols = []
    for i in range(5):
        col_name = 'account_id_' + str(i)
        account_id_cols.append(col_name)
    for col_name in account_id_cols:                
        players = players.union(set(outcome[col_name].values))        
    players = players - set([0])
    players = {player: {} for player in players}
    
    outcome.sort_values(by=['match_id', 'rad'], inplace=True)    
    n_matches = len(outcome)//2
    # record data
    for i in range(n_matches):
        if i % 10000 == 0:
            print('{:.2%}'.format(i/n_matches))
            display.clear_output(wait=True)
        row_dire = i*2
        row_rad = i*2+1        
        team_dire = set(outcome.loc[row_dire, account_id_cols].values)-set([0])
        team_rad = set(outcome.loc[row_rad, account_id_cols].values)-set([0])
        
        # record team mate info
        for team in [team_dire, team_rad]:
            for pair in itertools.combinations(team, 2):
                for selected_id in [0, 1]:                    
                    other_id = (selected_id+1)%2
                    selected = pair[selected_id]
                    other = pair[other_id]

                    if other not in players[selected]:
                        players[selected][other] = [0, 0]
                    players[selected][other][0] += 1
        
        # record opponent info
        for id_rad in team_rad:
            for id_dire in team_dire:
                if id_dire not in players[id_rad]:
                    players[id_rad][id_dire] = [0, 0]
                if id_rad not in players[id_dire]:
                    players[id_dire][id_rad] = [0, 0]
                players[id_dire][id_rad][1] += 1
                players[id_rad][id_dire][1] += 1
        
    return players

In [5]:
folder = 'D:/OMSCS/CSE6242/Project/Plot1/data/'
file = 'match_outcome'

In [6]:
match_outcome = pd.read_csv(folder+file+'.csv').drop('Unnamed: 0', axis=1)

In [9]:
player_net = get_player_map(match_outcome)

99.53%


In [130]:
with open(folder+'player_net', 'wb') as handle:
    pickle.dump(player_net, handle, protocol=pickle.HIGHEST_PROTOCOL)